#### import neccesary libraries

In [2]:
import pandas as pd
import numpy as np
from fpdf import FPDF
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
import os
import numpy as np
from tqdm import tqdm
import time
import datetime

#### read dataframe

In [9]:
dataframe = pd.read_csv("formated_frame.csv")

___

#### set wave and day

In [5]:
wave_num = 2

In [6]:
day_num = 7

In [7]:
day_data = dataframe.loc[dataframe["wave"] == wave_num].loc[dataframe["day"] == day_num]

____

because of gmail account shut down, from wave 1, day 3 we had to replace an account

In [45]:
for i in range(day_data.shape[0]):
    if day_data.loc[i, "email_from"] == "molnarpanna85@gmail.com":
        day_data.loc[i, "email_from"] = "molnar.panna84@gmail.com"

___

### send emails

In [46]:
day_data["sent_time"] = None

In [48]:
for i in tqdm(range(day_data.shape[0])):
    # Data
    row = day_data.loc[i].to_dict()

    # Inputs
    email_from = row["email_from"]
    email_password = "ezegyjelszo"
    email_subject = row["subject"]
    email_to = row["email_to"]

    # Message
    msg = MIMEMultipart()
    msg["From"] = email_from
    msg["Subject"] = email_subject
    msg["To"] = email_to
    msg.attach(MIMEText(row["body"], "plain"))
    text = msg.as_string()

    # Server
    try:
        server = smtplib.SMTP("smtp.gmail.com", 587)
        server.starttls()

        server.login(email_from, email_password)
        server.sendmail(email_from, email_to, text)

        server.quit()

        day_data.loc[i, "sent_time"] = datetime.datetime.now()
        day_data.loc[i, "status"] = "Mail sent"

    except Exception as e:
        day_data.loc[i, "status"] = "Mail not sent"
        print(i, repr(e))

    time.sleep(2)

100%|██████████| 171/171 [10:01<00:00,  3.52s/it]


#### write sent daily data to local

In [49]:
day_data.to_csv("w2_d7.csv", index=False)

___

## ping sender

read the corresponding dataframe with already attached answers

In [2]:
wave_2 = pd.read_excel("w2_answers_0828.xlsx")#.drop(columns="Unnamed: 0")

In [ ]:
ping_day_num = 7

In [12]:
send_ping = (
    wave_2.loc[wave_2["day"] == ping_day_num]
    .loc[wave_2["response"].isnull()]
    .loc[wave_2["ping"] == True]
).reset_index(drop=True)

In [13]:
for i in tqdm(range(send_ping.shape[0])):
    # Data
    row = send_ping.loc[i].to_dict()

    # Inputs
    email_from = row["email_from"]
    email_password = "ezegyjelszo"
    email_subject = row["subject"]
    email_to = row["email_to"]

    # Message
    msg = MIMEMultipart()
    msg["From"] = email_from
    msg["Subject"] = email_subject
    msg["To"] = email_to
    msg.attach(MIMEText(row["ping_body"], "plain"))
    text = msg.as_string()

    # Server
    try:
        server = smtplib.SMTP("smtp.gmail.com", 587)
        server.starttls()

        server.login(email_from, email_password)
        server.sendmail(email_from, email_to, text)

        server.quit()

        # get_index = wave2a.loc[wave2a["index"] == send_ping.loc[i, "index"]].index
        # wave2a.loc[get_index, "ping_sent"] = True

    except Exception as e:
        print(i, repr(e))
        # get_index = wave2a.loc[wave2a["index"] == send_ping.loc[i, "index"]].index

    time.sleep(2)

100%|██████████| 36/36 [01:55<00:00,  3.20s/it]


In [14]:
send_ping.to_csv("ping_d7.csv", index=False)